## Load the libs

In [1]:
from pdf2image import convert_from_path
import cv2
import numpy as np
import imutils
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from keras.models import sequential, load_model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

## Load PDF for operation

In [2]:
# P_ID_file = 'B145-101-1140.pdf'
# img = convert_from_path(P_ID_file)
# for i in range(len(img)):
#     img[i].save(r"\Users\Dhrupad\Desktop\Leven\P and ID\Untitled Folder\Demo\dem_2\p_id_image"+str(i)+".jpg","JPEG")

### Load the image and process the image 140 140 section

In [2]:
p_id_img = cv2.imread("pid.png")

In [3]:
# %matplotlib
# plt.imshow(p_id_img)

In [5]:
# Cropping an image
# cropped_image = p_id_img[200:6300, 880:7500]
# Display cropped image
%matplotlib
plt.imshow(p_id_img)

Using matplotlib backend: Qt5Agg


In [6]:
# find all the 'black' shapes in the image
lower = np.array([0, 0, 0])
upper = np.array([50, 50, 50])
shapeMask = cv2.inRange(cropped_image, lower, upper)

#### object detection 

In [7]:
cnts = cv2.findContours(shapeMask.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
print("Founded {} black shapes".format(len(cnts)))

Founded 181 black shapes


In [8]:
for i in range(len(cnts)):
    y_max,x_max = np.max(cnts[i],axis=0)[0]
    y_min,x_min = np.min(cnts[i],axis=0)[0]
    x_c = int((x_min+x_max)/2)
    y_c = int((y_min+y_max)/2)
    cv2.imwrite("p_id_image"+str(i)+"contour.jpg", 
                p_id_img[x_c-70:x_c+70,y_c-70:y_c+70])

#### image process and loading in nn model

In [26]:
ip_img=[]
for i in range(150):
    if os.path.isfile("p_id_image"+str(i)+"contour.jpg"):
        img = cv2.imread("p_id_image"+str(i)+"contour.jpg")
    #     print(i)
        r_img = cv2.resize(img,(100,100))
        grayImage = cv2.cvtColor(r_img, cv2.COLOR_BGR2GRAY)
    #     print(grayImage.shape)
        g_img = grayImage.reshape(100,100,1)
        n_img = cv2.bitwise_not(g_img)
        ip_img.append(n_img)

In [27]:
nsp= np.array(ip_img)
d1,d2,d3=nsp.shape

In [28]:
trn_model = tf.keras.models.load_model('eng_drwing.model')
perd = trn_model.predict(nsp.reshape(d1,d2,d3,1))

In [29]:
perd.shape

(11, 30)

In [30]:
label_names = ['Arrowhead', 'Arrowhead + Triangle', 'Box', 'Continuity Label',
       'Control', 'Control Valve Globe', 'DB&BBV', 'DB&BBV + Valve Check',
       'DB&BPV', 'ESDV Valve Ball', 'ESDV Valve Butterfly',
       'ESDV Valve Slab Gate', 'Exit to Atmosphere', 'Flange + Triangle',
       'Flange Joint', 'Flange Single T-Shape', 'Injector Point',
       'Reducer', 'Rupture Disc', 'Sensor', 'Spectacle Blind', 'Triangle',
       'Valve', 'Valve Angle', 'Valve Ball', 'Valve Butterfly',
       'Valve Check', 'Valve Globe', 'Valve Plug', 'Valve Slab Gate']

In [31]:
product = nsp.reshape(nsp.shape[0],100,100)
fig, axis = plt.subplots(4, 2, figsize=(12, 12))
for i,ax in enumerate(axis.flat):
    l = i
    ax.imshow(product[l],cmap='binary')
    ax.set(title=f'predicted_symbol_{label_names[perd[l].argmax()]}');
    

In [67]:
perd[5].argmax()

17

#### Store the symbols in txt

In [16]:
file = open('Symboles_1.txt','w')
for i in range(len(perd)):
    file.write('\nDetected symbole is:'+str(label_names[perd[i].argmax()]))
file.close()